In [1]:
import os
import json
import numpy as np
import itertools
import re
from collections import defaultdict
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
keypoints_dict={
        0: "nose",
        1: "left_eye",
        2: "right_eye",
        3: "left_ear",
        4: "right_ear",
        5: "left_shoulder",
        6: "right_shoulder",
        7: "left_elbow",
        8: "right_elbow",
        9: "left_wrist",
        10: "right_wrist",
        11: "left_hip",
        12: "right_hip",
        13: "left_knee",
        14: "right_knee",
        15: "left_ankle",
        16: "right_ankle"
    }

In [3]:
def create_bbox(lst):
    res = []
    res.append([lst[1][0], lst[0][1]])
    res.append([element1 - element2 for (element1, element2) in zip(lst[1], lst[0])])
    res = list(itertools.chain(*res))
    return res

In [49]:
def coco_converter(label, area_info):
    image_id = re.split('/|\\\\',label['imagePath'])[-1].split('.')[0]
    image_id = str(int(''.join(filter(str.isdigit, image_id.replace('_', '0')))))
    for area_id in (area_info['images']):
        if area_id['file_name'].split('/')[-1].split('.')[0] == image_id:
            area = area_info['annotations'][int(area_id['id'])]['area']
            bbox = area_info['annotations'][int(area_id['id'])]['bbox']
    keypoints = dict.fromkeys(keypoints_dict.values())
    #bbox = defaultdict(list)
    for i in label['shapes']:
        # if i['label'] == 'bbox':
        #     bbox["bbox"]=(create_bbox(i['points']))
        for x in keypoints_dict.values():
            if i['label'] == x:
                key = list(itertools.chain(*i['points']))
                key.append(i['group_id'])
                keypoints[i['label']] = key
    for k,v in keypoints.items():
        if v[2] == 0:
            keypoints[k] = [0,0,0]
    keypoints_lst = []
    for i in keypoints.values():
        keypoints_lst.append(i)
    keypoints_lst = list(itertools.chain(*keypoints_lst))
    images = {}
    images.update(({"file_name": image_id+'.png'}))
    images.update(({"id": int(image_id)}))
    images.update({"width": 640})
    images.update({"height": 480})
    annotations = {}
    annotations.update({"bbox": bbox})
    annotations.update({"area" :area})
    annotations.update({"id" : int(image_id)})
    annotations.update({"image_id" : int(image_id)})
    annotations.update({"keypoints": keypoints_lst})
    annotations.update({"category_id": 1})
    annotations.update({"iscrowd": 0})
    annotations.update({"num_keypoints": 17})
    return images, annotations

In [50]:
f = open('/home/hugo/Desktop/test/annotations.json')
area_info = json.load(f)

In [56]:
#train_set
converted_label = {"annotations": [], "images": [], "categories": []}
for file in (os.listdir(os.getcwd())):
    if file.endswith('.json'):
        f = open(os.path.join(os.getcwd(), file))
        label = json.load(f)
        images, annotations = coco_converter(label, area_info)
        converted_label['images'].append(images)
        converted_label['annotations'].append(annotations)
categories = {}
categories.update({"supercategory": "person"})
categories.update({"name": "person"})
categories.update({"id": 1})
categories.update({"keypoints": ["nose", "left_eye", "right_eye", "left_ear", "right_ear", "left_shoulder", "right_shoulder", "left_elbow", 
"right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", "right_knee", "left_ankle", "right_ankle"]})
categories.update({"skeleton": [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13], [6,7],[6,8],
        [7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]})
converted_label['categories'].append(categories)
converted_label = json.dumps(converted_label) #need to save it manually to json file 

In [ ]:
converted_label

In [59]:
test_path = '/home/hugo/Desktop/deep-high-resolution-net.pytorch-master/data/ours/annotations/test_annotations'
converted_label = {"annotations": [], "images": [], "categories": []}
for file in (os.listdir(test_path)):
    if file.endswith('.json'):
        f = open(os.path.join(test_path, file))
        label = json.load(f)
        images, annotations = coco_converter(label, area_info)
        converted_label['images'].append(images)
        converted_label['annotations'].append(annotations)
categories = {}
categories.update({"supercategory": "person"})
categories.update({"name": "person"})
categories.update({"id": 1})
categories.update({"keypoints": ["nose", "left_eye", "right_eye", "left_ear", "right_ear", "left_shoulder", "right_shoulder", "left_elbow", 
"right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", "right_knee", "left_ankle", "right_ankle"]})
categories.update({"skeleton": [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], 
    [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], 
    [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], 
    [3, 5], [4, 6], [5, 7]]})
converted_label['categories'].append(categories)
converted_label = json.dumps(converted_label) #need to save it manually to json file 
# out_file = open(os.path.join(os.path.dirname(os.getcwd()), "converted.json"), "w")

In [ ]:
converted_label

In [26]:
from matplotlib import image
path = "/home/hugo/Desktop/deep-high-resolution-net.pytorch-master/data/syrip/annotations/person_keypoints_train_infant.json"
f = open(path)
syrip = json.load(f)
syrip.keys()
ids = []
for img in syrip['images']:
    if img['file_name'] == 'train00001.jpg':
        kept = syrip['annotations'][0]['keypoints']
invisible_kept = []
for c, i in enumerate(kept):
    if i < 2:
        invisible_kept.append(c)

dict_keys(['images', 'annotations', 'categories'])

In [27]:
invisible_kept
kept

[30, 31, 32]

[461.6506652832031,
 645.4415283203125,
 2.0,
 588.2577514648438,
 601.3251342773438,
 2.0,
 381.0824890136719,
 538.0277709960938,
 2.0,
 851.0634155273438,
 505.4200439453125,
 2.0,
 355.3552485108878,
 452.21578947368425,
 2.0,
 905.0815643003616,
 674.980701754386,
 2.0,
 254.7517397389579,
 671.3877192982457,
 2.0,
 996.8534545898438,
 1034.8162841796875,
 2.0,
 28.393845002115768,
 1052.243859649123,
 2.0,
 996.8534545898438,
 1351.3033447265625,
 2.0,
 -45.0,
 -45.0,
 0.0,
 868.3280639648438,
 1324.4498291015625,
 2.0,
 434.7945861816406,
 1405.01025390625,
 2.0,
 1167.3692835986071,
 1314.5315789473684,
 2.0,
 536.4639892578125,
 1663.9539794921875,
 2.0,
 1119.6240234375,
 1585.3118896484375,
 2.0,
 1002.6082763671875,
 1876.8634033203125,
 2.0]